<a href="https://colab.research.google.com/github/riccardo247/star-GPT/blob/main/offchart_star_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Download all needd packages**


> package GPTNeo is needed with all the requirements



In [1]:
#@title Setup
%tensorflow_version 2.x
!git clone https://github.com/EleutherAI/GPTNeo
%cd GPTNeo
!pip3 install -q -r requirements.txt
pretrained_model = None
dataset = None

Cloning into 'GPTNeo'...
remote: Enumerating objects: 3747, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 3747 (delta 77), reused 127 (delta 69), pack-reused 3609
Receiving objects: 100% (3747/3747), 1.44 MiB | 9.57 MiB/s, done.
Resolving deltas: 100% (2167/2167), done.
/content/GPTNeo
     |████████████████████████████████| 368kB 6.4MB/s 
     |████████████████████████████████| 14.2MB 222kB/s 
     |████████████████████████████████| 112kB 55.3MB/s 
     |████████████████████████████████| 394.7MB 39kB/s 
     |████████████████████████████████| 3.4MB 38.3MB/s 
     |████████████████████████████████| 2.9MB 30.8MB/s 
     |████████████████████████████████| 1.5MB 28.1MB/s 
     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 184kB 40.0MB/s 
     |████████████████████████████████| 2.2MB 39.6MB/s 
     |████████████████████████████████| 1.0MB 39.3MB/s 
     |███████████████████████

**Gopogle authentication** could be needed to access the Google bucket with weights

In [8]:
from google.colab import auth
auth.authenticate_user()
!gcloud init1

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'
core:
  account: riccardo.giacomelli@gmail.com

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

Choose the account you would like to use to perform operations for 
this configuration:
 [1] riccardo.giacomelli@gmail.com
 [2] Log in with a new account
Please enter your numeric choice:  1

You are logged in as: [riccardo.giacomelli@gmail.com].

Pick cloud proje

**Configuration**


> The following are configurations files for the model and data. They do not need to be modified



In [20]:
%%writefile configs/GPT3_XL_bakeoff.json

{
    "n_head": 16,
    "n_vocab": 50257,
    "embed_dropout": 0,
    "lr": 0.0002,
    "lr_decay": "cosine",
    "warmup_steps": 3000,
    "beta1": 0.9,
    "beta2": 0.95,
    "epsilon": 1e-8,
    "opt_name": "adam",
    "weight_decay": 0,
    "train_batch_size": 1,
    "attn_dropout": 0,
    "train_steps": 744136,
    "lr_decay_end" : 300000,
    "eval_steps": 20,
    "predict_steps":1,
    "res_dropout": 0,
    "eval_batch_size": 1,
    "predict_batch_size": 1,
    "iterations": 100,
    "n_embd": 2048,
    "datasets": [["sst3_shortbake_label_pred"]], 
    "model_path": "gs://bigbird-freefly/neo_gpt3/gpt3_XL/the-eye.eu/public/AI/gptneo-release/all_balanced_train_b",
    "n_ctx": 2048,
    "n_layer": 24,
    "scale_by_depth": true,
    "scale_by_in": false,
    "attention_types" :  [[["global", "local"],12]],
    "mesh_shape": "x:1,y:8",
    "layout" : "batch:x,memory_length:y,embd:y",
    "activation_function": "gelu",
    "recompute_grad": true,
    "gradient_clipping": 1.0,
    "tokens_per_mb_per_replica": 2048,
    "precision": "bfloat16",
    "padding_id" : 50257,
    "eos_id" : 50256
}

Overwriting configs/GPT3_XL_bakeoff.json


In [21]:
%%writefile configs/dataset_configs/sst3_shortbake_label_pred.json
{
    "n_vocab": 50257,
    "path": "gs://bigbird-freefly/sst3/bakeoff_entry//sst*.tfrecords", 
    "eval_path": "gs://bigbird-freefly/sst3/dev//imbd*.tfrecords",
    "tokenizer_is_pretrained": true,
    "tokenizer_path": "gpt2",
    "eos_id" : 50256,
    "padding_id": 50257
}

Overwriting configs/dataset_configs/sst3_shortbake_label_pred.json


In [11]:
!gsutil cp gs://bigbird-freefly/sst3/files/sample.py ./

Copying gs://bigbird-freefly/sst3/files/sample.py...
/ [1 files][  9.1 KiB/  9.1 KiB]                                                
Operation completed over 1 objects/9.1 KiB.                                      


**Review examples**


> The following code cell writes to a txt a review example. The text should be formatted in 3 lines as:


> This is the review:
...put the text of the review in this line...
sentiment is:





In [22]:
%%writefile review_11820-00001.txt
This is a review:
Enough similarities to Gymkata and Howie Long 's Firestorm that my fingernails instinctively crawled towards my long-suffering eyeballs .
sentiment is:

Overwriting review_11820-00001.txt


In [6]:
%%writefile review_11830-00001.txt
This is a review:
While the mystery surrounding the nature of the boat 's malediction remains intriguing enough to sustain mild interest , the picture refuses to offer much accompanying sustenance in the way of characterization , humor or plain old popcorn fun .
sentiment is:

Overwriting review_11830-00001.txt


**Sentimen prediction:**
The follosing is running prediction for the txt file and output the result to the cell.

>The first time it will take 10 minutes because it has to download the model weights. Second time will be 1-2 minutes. This notebook is just to show how it works but it is very slow!
Output will be the input ending in sentiment like this:
This is a review:
While the mystery surrounding the nature of the boat 's malediction remains intriguing enough to sustain mild interest , the picture refuses to offer much accompanying sustenance in the way of characterization , humor or plain old popcorn fun .
sentiment is: **negative**




In [23]:
review_file ='review_11820-00001.txt'
!python3 main.py --model GPT3_XL_bakeoff --steps_per_checkpoint 100 --tpu colab --predict --prompt $review_file

2021-04-21 20:45:26.644600: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
Current step 736911
Saving config to gs://bigbird-freefly/neo_gpt3/gpt3_XL/the-eye.eu/public/AI/gptneo-release/all_balanced_train_b
2021-04-21 20:45:35.333094: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-21 20:45:35.334145: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-21 20:45:35.346532: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-21 20:45:35.346619: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (945b4dd6c79b): /proc/driver/nvidia/version does not e